In [24]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install mojimoji
!pip install scipy
!pip install nltk
!pip install numpy
!pip install textcleaner

#!apt-get -q -y install mecab libmecab-dev file
#!git clone --depth 1 https://github.com/neologd/mecab-unidic-neologd.git
#!echo yes | mecab-unidic-neologd/bin/install-mecab-unidic-neologd -n

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip'


In [25]:
import random
import sys
import numpy as np
from scipy.sparse import lil_matrix
import MeCab 
import unicodedata
import pandas as pd
import glob
import math
import matplotlib.pyplot as plt
import time
import datetime
import pytz
import os
import more_itertools
import re

ModuleNotFoundError: No module named 'numpy'

In [ ]:
"""関数のインポート"""
from TopicModelFunctions import *

ModuleNotFoundError: No module named 'numpy'

In [ ]:
  """
  文書ごとのトピック分布を計算 display=Trueで出力
  """

def show_topic(docs, K, titles=None, texts=None, category_doc=None, display=True):
  wakati=wakati_docs(texts)
  CORPUS, WORD_TO_ID, ID_TO_WORD = preprocess(texts)
  term_list = np.zeros((len(ID_TO_WORD), K))
  cate_list = np.zeros((len(types), K))
  theta = np.zeros((len(docs), K), dtype=float)
  total=0
  for i in range(len(docs)):
    for j in range(len(docs[i])):
      term_list[CORPUS[total+j]][docs[i][j]] = term_list[CORPUS[total+j]][docs[i][j]]+1
    total += len(docs[i])
  if category_doc:
    for i in range(len(category_doc)):
      for j in range(len(category_doc[i])):
        if type_list[i][j] != 0:
          try:
            cate_list[j][category_doc[i][j]] += 1
          except Exception:
            print(category_doc[i])
  cnt = list(range(K))
  for num in range(K):
   cnt[num] = 0
  index = 0
  for idx in range(len(docs)):
    doc = docs[idx]
    if titles is not None:
      title = titles[idx]
    else:
      title = ""
    if texts:
      text = texts[idx]
    else:
      text = ""
    if category_doc:
      category = []
      for i in range(len(category_doc[idx])):
        if category_doc[idx][i] != 0:
          try:
            category.append(types[i])
          except Exception:
            print(category_doc[idx][i])
    else:
      category = ""
    if display:
      print("doc", idx, title, category, ":", text)
    index += 1
    for val in doc:
      for num in range(K):
        if(val==num):
          cnt[num] += 1
    for num in range(K):
      theta[idx][num] = cnt[num]/len(doc)
      if display:
        print("Topic", num, "=", theta[idx][num]*100, "%")
      cnt[num] = 0
  if category_doc:
    return term_list, cate_list, theta
  else:
    return term_list, theta


In [ ]:
  """
  トピックごとの頻出単語を降順に取得 display=trueで出力
  """
def topic_term(topic, K, ID_TO_WORD, category_topic=None, display=True, term_num=100):
  freq_words = []
  for i in range(K):
    arr = topic[:,i]
    ans = np.argsort(arr)[::-1]
    ans_term = []
    for j in range(term_num):
      ans_term.append(ID_TO_WORD[ans[j]])
    if category_topic is not None:
      arr_c = category_topic[:,i]
      ans_c = np.argsort(arr_c)[::-1]
      ans_term_c = []
      for j in range(len(types)):
        ans_term_c.append(types[ans_c[j]])

      ans_term_c = np.array(ans_term_c)
    else:
      ans_term_c = ""
    if display:
      print("Topic", i, ":", ans_term_c)
    ans_term = np.array(ans_term)
    if display:
      print(ans_term)
    freq_words.append(ans_term)
  return freq_words

In [ ]:
"""共起頻度の算出"""
def co_occurrence(term1, term2, BoW, word2id):
  cnt = 0
  for i in range(len(BoW)):
    if BoW[i][word2id[term1]] >= 1 and BoW[i][word2id[term2]] >= 1:
      cnt += 1
  return float(cnt)

""" 対数条件付き確率を用いたCoherence(Mimno, 2011) """
def calculate_coherence(freq_words, word2id, BoW):
  Coh = np.zeros((len(freq_words)), dtype=float)
  for topic_idx in range(len(freq_words)):
    for i in range(len(freq_words[topic_idx])):
      for j in range(len(freq_words[topic_idx])):
        if i <= j:
          break
        else:
          term1 = str(freq_words[topic_idx][i])
          term2 = str(freq_words[topic_idx][j])
          if word2id[term2] < 0 or word2id[term2] > len(word2id) or co_occurrence(term2, term2, BoW, word2id)==0:
            continue
          Coh[topic_idx] += math.log((co_occurrence(term1, term2, BoW, word2id)+1)
          /co_occurrence(term2, term2, BoW, word2id))
  return Coh

"""　最尤推定によるΦ（文書毎のトピック分布）の計算 """
def calculate_phi(docs, fit_docs, K, word2id): 
  phi = np.zeros((len(word2id), K), dtype=float)
  for idx in range(len(docs)):
    doc = docs[idx]
    fit_doc = fit_docs[idx]
    for i in range(len(fit_doc)):
      for num in range(K):
        if fit_doc[i] == num:
          phi[word2id[doc[i]]][num] += 1
  sum_phi = np.sum(phi, axis=0)
  for i in range(len(word2id)):
    for j in range(K):
      phi[i][j] = (phi[i][j])/sum_phi[j]
  return phi

In [ ]:
""" perplexityの計算（岩田の青本）"""
def calculate_perplexity(docs, theta, phi, word2id): 
  Per = 0
  K = theta.ndim
  text_len_sum = 0
  likelihood = 0
  for idx in range(len(docs)):
    doc = docs[idx]
    text_len_sum += len(doc)
    sum1 = 1.0
    for i in range(len(doc)):
      sum0 = 0.0
      for num in range(K):
        try:
          sum0 += theta[idx][num]*phi[word2id[doc[i]]][num]
        except Exception:
          word2id[doc[i]] = len(word2id)
      sum1 *= sum0
    if sum1 == 0:
      continue
    likelihood += math.log(sum1)
  Per = float(math.exp(-(likelihood/text_len_sum)))
  return Per

In [ ]:
"""LDAとJTM2の比較"""
def compare_LDA_JTM(K0, alpha0, beta0, max_iter0):
  start = time.time()

  term_num = 100
  model1 = JTM(K=K0, alpha=alpha0, beta=beta0, max_iter=max_iter0)
  model2 = JTM(K=K0, alpha=alpha0, beta=beta0, max_iter=max_iter0)

  model1.fit(X,V)
  topic1, theta1 = show_topic(model1.Z[0], K0, titles=df_title, texts=Df_train, display=None)
  freq1 = topic_term(topic1, K0, ID_TO_WORD, display=None, term_num=term_num)
  coh1 = calculate_coherence(freq1, WORD_TO_ID, BoW)

  model2.fit(X1,V1)
  topic2, category_topic, theta2 = show_topic(model2.Z[0], K0, titles=list(df_title), 
                   texts=Df_train, category_doc=model2.Z[1], display=None)
  freq2 = topic_term(topic2, K0, ID_TO_WORD, category_topic=category_topic, 
                     display=None, term_num=term_num)
  coh2 = calculate_coherence(freq2, WORD_TO_ID, BoW)
  
  nkv1 = []
  ndk1 = []
  nkv2 = []
  ndk2 = []
  for v in range(V[0]):
   nkv1.append(model1.nkv[0][:,v])
  for v in range(V2[0]):
    nkv2.append(model2.nkv[0][:,v])
  for k in range(K0):
    ndk1.append(model1.ndk[0][:,k])
    ndk2.append(model2.ndk[0][:,k])
  theta1 = (ndk1/model1.nd[0]).T
  phi1 = nkv1/model1.nk[0]
  theta2 = (ndk2/model2.nd[0]).T
  phi2 = nkv2/model2.nk[0]

  per1 = calculate_perplexity(wakati_test, theta1, phi1, WORD_TO_ID)
  per2 = calculate_perplexity(wakati_test, theta2, phi2, WORD_TO_ID)

  ans = [int(K0), alpha0, beta0, int(max_iter0), 
         sum(coh1)/K0, np.std(coh1), sum(coh2)/K0, np.std(coh2), per1, per2]
  ans = pd.Series(data=ans, index=columns0)
  display(ans)
  elapsed_time = int(time.time() - start)
  minute, second = time2MinSec(elapsed_time)
  print(f"elapsed_time : {minute}[min] {second}[sec]")

  return pd.Series(data=ans, index=columns0)

In [ ]:
"""JTM1とJTM2の比較"""
def compare_JTM_JTM(K0, alpha0, beta0, max_iter0):
  start = time.time()

  term_num = 100
  model1 = JTM(K=K0, alpha=alpha0, beta=beta0, max_iter=max_iter0)
  model2 = JTM(K=K0, alpha=alpha0, beta=beta0, max_iter=max_iter0)

  model1.fit(X1,V1)
  topic1, category_topic, theta1 = show_topic(model1.Z[0], K0, titles=list(df_title), 
                   texts=Df_train, category_doc=model1.Z[1], display=None)
  freq1 = topic_term(topic1, K0, ID_TO_WORD, category_topic=category_topic, 
                     display=None, term_num=term_num)
  coh1 = calculate_coherence(freq1, WORD_TO_ID, BoW)

  model2.fit(X2,V2)
  topic2, category_topic, theta2 = show_topic(model2.Z[0], K0, titles=list(df_title), 
                   texts=Df_train, category_doc=model2.Z[1], display=None)
  freq2 = topic_term(topic2, K0, ID_TO_WORD, category_topic=category_topic, 
                     display=None, term_num=term_num)
  coh2 = calculate_coherence(freq2, WORD_TO_ID, BoW)
  
  nkv1 = []
  ndk1 = []
  nkv2 = []
  ndk2 = []
  for v in range(V[0]):
   nkv1.append(model1.nkv[0][:,v])
  for v in range(V2[0]):
    nkv2.append(model2.nkv[0][:,v])
  for k in range(K0):
    ndk1.append(model1.ndk[0][:,k])
    ndk2.append(model2.ndk[0][:,k])
  theta1 = (ndk1/model1.nd[0]).T
  phi1 = nkv1/model1.nk[0]
  theta2 = (ndk2/model2.nd[0]).T
  phi2 = nkv2/model2.nk[0]

  per1 = calculate_perplexity(wakati_test, theta1, phi1, WORD_TO_ID)
  per2 = calculate_perplexity(wakati_test, theta2, phi2, WORD_TO_ID)

  ans = [int(K0), alpha0, beta0, int(max_iter0), 
         sum(coh1)/K0, np.std(coh1), sum(coh2)/K0, np.std(coh2), per1, per2]
  ans = pd.Series(data=ans, index=columns1)
  display(ans)
  elapsed_time = int(time.time() - start)
  minute, second = time2MinSec(elapsed_time)
  print(f"elapsed_time : {minute}[min] {second}[sec]")

  return pd.Series(data=ans, index=columns1)

In [ ]:
"""coherence, perplexity描画用の関数"""
def plot_coherence(title, ave, std, Param):
  Coh_max = Param[0]
  Coh_min = Param[1]
  Coh_range = Param[2]
  Per_max = Param[3]
  fig, ax2 = plt.subplots(1,1,figsize=(12,4.5),tight_layout=True)
  ax1 = ax2.twinx()
  ax1.plot(coh_df["K"],ave,linestyle="solid",color="k",marker="^",label="Coherence(Average)")
  ax2.plot(coh_df["K"],std,linestyle="dashdot",color="k",marker="v",label="Perplexity")
  ax1.set_ylim(Coh_min-(Coh_range/10), Coh_max+(Coh_range/10))
  ax1.set_ylabel("Coherence")
  ax2.set_ylim(1,Per_max*1.1)
  ax2.set_ylabel("Perplexity")
  handler1, label1 = ax1.get_legend_handles_labels()
  handler2, label2 = ax2.get_legend_handles_labels()
  ax1.legend(handler1+handler2,label1+label2,borderaxespad=0)
  ax1.grid(True)
  plt.title(title)
  return fig

"""average, standard deviation描画用の関数"""
def plot_ave_or_std(title, data1, data2, Param, Ave=None, Std=None):
  Coh_max = Param[0]
  Coh_min = Param[1]
  Coh_range = Param[2]
  Std_max = Param[3]
  fig, ax1 = plt.subplots(1,1,figsize=(12,4.5),tight_layout=True)
  if Ave:
    ax1.plot(coh_df["K"],data1,linestyle="solid",marker="o", color="orange",
           label="Latent Dirichlet Allocation")
    ax1.plot(coh_df["K"],data2,linestyle="solid",marker="v", color="lightblue",
           label="Joint Topc Model")
    ax1.set_ylim(Coh_min-(Coh_range/10), Coh_max+(Coh_range/10))
  if Std:
    ax1.bar(coh_df["K"],data1,color="orange", alpha=0.5, width=0.5,
           label="Latent Dirichlet Allocation")
    ax1.bar(coh_df["K"],data2,color="skyblue", alpha=0.5, width=0.5,
           label="Joint Topic Model", align='edge')
    ax1.set_ylim(0,Std_max*1.1)
  ax1.set_ylabel("Value")
  handler1, label1 = ax1.get_legend_handles_labels()
  ax1.legend(handler1,label1,borderaxespad=0,loc=2)
  ax1.grid(True)
  plt.title(title)
  return fig

In [ ]:
"""
Webシラバスのスクレイピング結果読み込みとデータフレームへの落とし込み
scrapiんg_result.csvは同階層に置いておくこと
"""
df = pd.read_csv(r"mie_scraping_result.csv")
df_valuate = pd.concat([df['成績評価方法と基準']], axis=0)
df_abstract = df['授業の概要']

df_aim = df['学修の目的']
df_key = df['キーワード']
df_achivement = df['学修の到達目標']
df_resource = df['授業改善の工夫']
df3 = df[0::1] 
df4 = df['開講区分']
df_plan = df['学修内容']
df_author = df['担当教員']
df_title = df['授業科目名']
df_types = df['授業の方法']
names = ["教養教育・教養統合科目・現代科学理解", "教養教育・教養統合科目・地域理解・日本理解", 
         "教養教育・教養統合科目・国際理解・現代社会理解"]
#今回は英語シラバスを除外するので外国人講師を割愛
ignore_authors = ["サコラヴスキー", "ロバート", "リチャーズ", "マクダニエル", "ジェフ", 
           "ジョンストン", "ブライアン", "ジュネジャ"]

for i in range(len(df3)): # 英語講師のシラバスは割愛
  for author in ignore_authors:
    if(author in str(df_author[i])):
      df_valuate[i] = np.nan
      df_abstract[i] = np.nan
      df_aim[i] = np.nan
      df_achivement[i] = np.nan
      df_plan[i] = np.nan
      df_title[i] = np.nan
      df_resource[i] = np.nan
      df_key[i] = np.nan
  if(True): # 三重大学の授業科目をnamesに限定する場合はTrue
    if type(df4[i]) == float or not df4[i].replace(' ', '') in names:
        df_valuate[i] = np.nan
        df_abstract[i] = np.nan
        df_aim[i] = np.nan
        df_achivement[i] = np.nan
        df_plan[i] = np.nan
        df_resource[i] = np.nan
        df_title[i] = np.nan
        df_key[i] = np.nan

df_valuate.index = list(range(len(df_valuate)))
df_abstract.index = list(range(len(df_abstract)))
df_aim.index = list(range(len(df_aim)))
df_achivement.index = list(range(len(df_achivement)))
df_resource.index = list(range(len(df_resource)))
df_plan.index = list(range(len(df_plan)))
df_title.index = list(range(len(df_title)))
df_types.index = list(range(len(df_types)))
df_key.index = list(range(len(df_key)))

print(len(df_title), len(df_valuate), len(df_aim), len(df_plan), len(df_resource))
df_tokyo = pd.read_csv(r"tokyo_scraping_result.csv")
df_tokyo = df_tokyo.dropna(subset=['授業科目名']).reset_index()
df_title = pd.concat([df_title, df_tokyo['授業科目名']])
df_plan = pd.concat([df_plan, df_tokyo['学修内容']])
df_resource = pd.concat([df_resource, df_tokyo['授業改善の工夫']])
df_valuate = pd.concat([df_valuate, df_tokyo['成績評価方法と基準']])
df_aim = pd.concat([df_aim, df_tokyo['学修の目的']])
df_title.index = list(range(len(df_title)))
df_valuate.index = list(range(len(df_valuate)))
df_abstract.index = list(range(len(df_abstract)))
df_aim.index = list(range(len(df_aim)))
df_achivement.index = list(range(len(df_achivement)))
df_resource.index = list(range(len(df_resource)))
df_plan.index = list(range(len(df_plan)))
print(len(df_title), len(df_valuate), len(df_aim), len(df_plan), len(df_resource))
dfAll = pd.concat([df_title, df_valuate, df_aim, df_plan, df_resource], axis = 1, ignore_index=False)
dfAll = dfAll.dropna(how="all")
dfAll.index = list(range(len(dfAll)))
dfAll.columns = ["title", "valuate", "aim", "plan", "resource"]
display(dfAll)

dfAll.to_csv("test.csv", encoding='utf_8_sig')

FileNotFoundError: ignored

In [ ]:
"""
提案手法
補助情報の候補となる授業形態全てについて総当たりで目的関数を最大化する組み合わせを見つける．
"""
def calculate_auxiliary_information(type_list, type_word2id, candidate_types):
  max_score = 0
  max_comb = []
  notlist=[]
  for comb in candidate_types:
    score = 0
    null_cnt = 0
    type_term = []
    for i in range(len(type_list)):
      type_term.append([])
      if type_list[i][type_word2id[comb]] > 0:
        type_term[i].append(comb)
      
      if len(type_term[i])==0:
        null_cnt += 1
      
    for i in range(len(type_list)-1):
      for j in range(i+1, len(type_list)):
        symmetric_difference = len(set(type_term[i]) ^ set(type_term[j]))
        intersection = len(set(type_term[i]) & set(type_term[j]))
        union = len(set(type_term[i]) | set(type_term[j]))
        score += (union - 2*intersection) / (union + 1)
    #score *= (len(type_list)-null_cnt)
    print(comb)
    print(score)
    if score==0:
      notlist.append(comb)
    
  candidate_types = [s for s in candidate_types if s not in notlist]
  # 2個以上の組み合わせ
  comb_list = [list(s) for s in more_itertools.powerset(candidate_types) if len(s) > 1]
  for comb in comb_list:
    score = 0
    null_cnt = 0
    type_term = []
    for i in range(len(type_list)):
      type_term.append([])
      for type_ in comb:
        if type_list[i][type_word2id[type_]] > 0:
          type_term[i].append(type_)
      
      if len(type_term[i])==0:
        null_cnt += 1
    
    for i in range(len(type_list)-1):
      for j in range(i+1, len(type_list)):
        symmetric_difference = len(set(type_term[i]) ^ set(type_term[j]))
        intersection = len(set(type_term[i]) & set(type_term[j]))
        union = len(set(type_term[i]) | set(type_term[j]))
        score += (union - 2*intersection) / (union + 1)
    #score *= (len(type_list)-null_cnt)
    if max_score < score:
      max_score = score
      max_comb = comb
      print(comb, score)

  return max_comb, max_score

In [ ]:
  """
  補助情報の候補
  """
types=["講義", "授業", "演習", "実習", "実験", "実地見学", "フィールドワーク", "筆記試験", "グループ", "ビデオ", "オムニバス", "主体的"]
jtm1_types=types
type_list = np.zeros((len(dfAll), len(types)), dtype=int)
df_all = []
df_train = []
df_test = []
#学習用文書データと検証用文書データとに分ける(perplexity計算用)
for i in range(len(dfAll)-1):
  df_train.append(f'{dfAll["valuate"][i]} {dfAll["aim"][i]} {dfAll["plan"][i]} {dfAll["resource"][i]}')
  df_test.append(f'{dfAll["title"][i]}')
  df_all.append(f'{dfAll["title"][i]} {dfAll["valuate"][i]} {dfAll["aim"][i]} {dfAll["plan"][i]} {dfAll["resource"][i]}')
df_train = pd.Series(df_train)
df_test = pd.Series(df_test)
df_all = pd.Series(df_all)
count_df(df_train, types, type_list)

In [ ]:
"""
補助情報の選定
"""
type_word2id = dict(zip(types, list(range(len(types)))))
max_term, max_score = calculate_auxiliary_information(type_list=type_list, type_word2id=type_word2id, candidate_types=types)

cnt = 0
for i, name in enumerate(types):
  if name not in max_term:
    type_list = np.delete(type_list, i-cnt, 1)
    cnt += 1
jtm2_types = max_term
print(types)
print(max_score)
print(type_list[6])

In [ ]:
"""
文書の分かち書き
"""
DfAll = list(df_all)
Df_train = list(df_train)
Df_test = list(df_test)
df_title = list(df_title)
try:
  CORPUS, WORD_TO_ID, ID_TO_WORD = preprocess(DfAll)
except Exception as e:
  print(e)
  CORPUS, WORD_TO_ID, ID_TO_WORD = preprocess(DfAll)
wakati = wakati_docs(DfAll)
wakati_train = wakati_docs(Df_train) # 学習用
wakati_test = wakati_docs(Df_test) # 検証用
wakati_train_cnt = 0
wakati_test_cnt = 0

for doc in wakati_train:
  wakati_train_cnt += len(doc)
for doc in wakati_test:
  wakati_test_cnt += len(doc)

"""
文書の特徴ベクトル（単語文書行列）を作成
"""
BoW = bag_of_words(wakati, WORD_TO_ID)
#未完成（出現頻度で絞る） 
#WORD_TO_ID, BoW = transform_word2id(WORD_TO_ID, BoW, 2, len(wakati)/2)

# 分かち書きした単語をidに変換
X,V = input_docs(wakati_train, WORD_TO_ID)
X1,V1 = input_docs_tags(wakati_train, type_list, WORD_TO_ID, jtm1_types)
X2,V2 = input_docs_tags(wakati_train, type_list, WORD_TO_ID, jtm2_types)
print(f"訓練文書長:{wakati_train_cnt}語　検証文書長:{wakati_test_cnt}語") # 語数の出力

In [ ]:
"""
データフレームの列名
"""
columns0 = ["K", "alpha", "beta", "max_iter",
            "Coh_ave(LDA)", "Coh_std(LDA)", "Coh_ave(JTM)", "Coh_std(JTM)", "Per(LDA)", "Per(JTM)"]
columns1 = ["K", "alpha", "beta", "max_iter",
            "Coh_ave(JTM1)", "Coh_std(JTM1)", "Coh_ave(JTM2)", "Coh_std(JTM2)", "Per(JTM1)", "Per(JTM2)"]

In [ ]:
"""
比較実験
"""

#トピック数の範囲を指定
K_list = [x for x in range(8, 12, 1)]  
#ハイパーパラメータ
alpha = 0.1
beta = 0.1
#最大反復回数
max_iter = 100
coh_df = pd.DataFrame(columns = columns0)
coh_df2 = pd.DataFrame(columns = columns1)

#実行時間の予測（経験と勘で計算）
expected_time = (((len(K_list))*max_iter)*(wakati_train_cnt)/6500+(wakati_test_cnt)/13000)*2.5
minute, second = time2MinSec(expected_time)
print(f"expected_time : {minute}[min] {second}[sec]")
print("")
start = time.time()
print("LDAとJTM2の比較\n")
for k in K_list:
  new_data = compare_LDA_JTM(k, alpha, beta, max_iter)
  coh_df = coh_df.append(new_data, ignore_index=True)
print("JTM1とJTM2の比較\n")
for k in K_list:
  new_data2 = compare_JTM_JTM(k, alpha, beta, max_iter)
  coh_df2 = coh_df2.append(new_data2, ignore_index=True)
  
print("")
total_time = int(time.time() - start)
minute, second = time2MinSec(total_time)
print(f"total_time : {minute}[min] {second}[sec]")
display(coh_df)
display(coh_df2)

In [ ]:
"""
LDAとJTM2の比較
"""
Coh_max = max([max(coh_df["Coh_ave(LDA)"]), max(coh_df["Coh_ave(JTM)"])])
Coh_min = min([min(coh_df["Coh_ave(LDA)"]), min(coh_df["Coh_ave(JTM)"])])
Coh_range = Coh_max - Coh_min

Std_max = max([max(coh_df["Coh_std(LDA)"]), max(coh_df["Coh_std(JTM)"])])
Per_max = max([max(coh_df["Per(LDA)"]), max(coh_df["Per(JTM)"])])
Per_range = Per_max - 1
Param_list = [Coh_max, Coh_min, Coh_range, Std_max]
Coh_Per = [Coh_max, Coh_min, Coh_range, Per_max]

fig1 = plot_coherence("Latent Dirichlet Allocation",coh_df["Coh_ave(LDA)"],coh_df["Per(LDA)"],Coh_Per)
fig2 = plot_coherence("Joint Topc Model",coh_df["Coh_ave(JTM)"],coh_df["Per(JTM)"],Coh_Per)
fig3 = plot_ave_or_std("Coherence(Average)", coh_df["Coh_ave(LDA)"], coh_df["Coh_ave(JTM)"], Param_list,Ave=True)
fig4 = plot_ave_or_std("Coherence(Standard deviation)", coh_df["Coh_std(LDA)"], coh_df["Coh_std(JTM)"], Param_list,Std=True)

Param_Per = [Per_max, 1, Per_range, Std_max]

fig5 = plot_ave_or_std("Perplexity", coh_df["Per(LDA)"], coh_df["Per(JTM)"], Param_Per,Ave=True)
#fig4.show()

In [ ]:
"""
JTM1とJTM2の比較
"""
Coh_max = max([max(coh_df2["Coh_ave(JTM1)"]), max(coh_df2["Coh_ave(JTM2)"])])
Coh_min = min([min(coh_df2["Coh_ave(JTM1)"]), min(coh_df2["Coh_ave(JTM2)"])])
Coh_range = Coh_max - Coh_min

Std_max = max([max(coh_df2["Coh_std(JTM1)"]), max(coh_df2["Coh_std(JTM2)"])])
Per_max = max([max(coh_df2["Per(JTM1)"]), max(coh_df2["Per(JTM2)"])])
Per_range = Per_max - 1
Param_list = [Coh_max, Coh_min, Coh_range, Std_max]
Coh_Per = [Coh_max, Coh_min, Coh_range, Per_max]

fig6 = plot_coherence("Joint Topc Model1",coh_df2["Coh_ave(JTM1)"],coh_df2["Per(JTM1)"],Coh_Per)
fig6 = plot_coherence("Joint Topc Model2",coh_df2["Coh_ave(JTM2)"],coh_df2["Per(JTM2)"],Coh_Per)
fig8 = plot_ave_or_std("Coherence(Average)", coh_df2["Coh_ave(JTM1)"], coh_df2["Coh_ave(JTM2)"], Param_list,Ave=True)
fig9 = plot_ave_or_std("Coherence(Standard deviation)", coh_df2["Coh_std(JTM1)"], coh_df2["Coh_std(JTM2)"], Param_list,Std=True)

Param_Per = [Per_max, 1, Per_range, Std_max]

fig10 = plot_ave_or_std("Perplexity", coh_df2["Per(JTM1)"], coh_df2["Per(JTM2)"], Param_Per,Ave=True)

In [ ]:
"""
google driveにlogを保存する用のコード
"""

"""
drive.mount('/content/drive')
root_dir = "/content/drive/MyDrive/log/"
dt_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
new_dir = f"{dt_now.month}-{dt_now.day}_{dt_now.hour}:{dt_now.minute}:{dt_now.second}"
os.mkdir(root_dir+new_dir)
dt_now = f"{dt_now.month}-{dt_now.day}"
f = open(root_dir+f"{new_dir}/{dt_now}memo.txt", "w")
f.write("12/25 count_dfの欠陥を修正（最後の二つの要素が何故かカウントできてなかった）\n")
f.write("1/6 補助情報の最適化を導入\nまた、科目データフレームに重要な欠陥（データのずれ）があったので修正。１２月の結果は間違っている\n")
f.write("訓練用:授業科目名，キーワード, 授業の概要，授業の目的，学修内容，成績評価基準 授業改善の工夫\n検証用:学修の到達目標\n")
f.write(f"訓練文書長:{wakati_train_cnt}語 検証文書長:{wakati_test_cnt}語\n")
f.write("Perplexityは検証用，周辺化ギブスサンプリングにおける積分消去でθ,φを計算\n 4月に大幅修正n")
for type_ in types:
  f.write(f"{type_}, ")
f.close()

figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10]
filename = f"K{min(K_list)}to{max(K_list)}_α{alpha}_β{beta}_iter{max_iter}"
coh_df.to_csv(root_dir+f"{new_dir}/{filename}_LDA_LDA.csv", encoding='utf_8_sig')
coh_df2.to_csv(root_dir+f"{new_dir}/{filename}_JTM_JTM.csv", encoding='utf_8_sig')
for i, fig in enumerate(figures):
  fig.savefig(root_dir+f"{new_dir}/fig{i+1}_{filename}.pdf")
"""